Замятина Екатерина 476 группа

$$ \min_{x\in Q} f(x) $$

Q - замкнутое выпуклое множество в бесконечномерном линейном пространсте, f(x) - выпуклая негладкая  функция

### Субградиентный метод с двойным усреднением (Subgradient Method with Double SimpleAveraging)

Описанный выше метод называется
субградиентным методом с двойным
усреднением, т.к. в нём применяется как усреднение субградиентов, так и
усреднение исходной последовательности (при $α_k ≡ 1$). 

$$x_t^+= arg\min_{x \in Q} {\frac{1}{A_t}(\sum_{i=0}^t a_i v(x_i),x) + \frac{\gamma}{2A_t} || x - x_0||^2}$$

$$ m_t = \frac{a_{t+1}}{A_{t+1}} \hspace{10mm} x_{t+1} = (1-m_t)x_t + m_t x_t^+ \hspace{10mm} \sum^t_{i=1} a_i = A_t \hspace{10mm} v(x) \in \partial f$$

Вся суть метода состоит в том, что мы свели функцию, не обладающую достаточной гладкостью (например модуль), к бесконечно гладкой функции, к которой уже можно применять любые ранее известные методы.

Метод сходится как : $f(x_k) - f(x^*) = O(\frac{1}{\sqrt{k}})$ при $a_k = 1, \hspace{10mm} A_t = t +1 \hspace{10mm} \gamma_k= \sqrt{k+1} $

Можно немного модифицирвать алгоритм выше и получить субградтиентный метод с простым двойным ускорением
$$x_t^+= arg\min_{x \in Q} {(\sum_{i=0}^t v(x_i),x) + \gamma || x - x_0||^2}$$

$$ x_{t+1} = \dfrac{t+1}{t+2}x_t + \dfrac{1}{t+2} x_t^+ \hspace{10mm} v(x) \in \partial f$$

Заметим, функция гладкая, а значит мы можем гарантировать, что градиент направлен по росту функции, кроме то , если у f(x) Липшивиц градиент,то можно воспользоваться проксимальной функцией для поиска argmin как индикаторной функцией проекции.

$$x_{t}^+= arg\min_{x \in Q} {(\sum_{i=0}^t v(x_i),x) + \gamma || x - x_0||^2} = arg\min_{x \in Q} g(x)$$ 
$$ \nabla g(x) = \sum_{i=0}^t v(x_i) + 2\gamma ( x - x_0) $$
$$ x* = \pi_Q(x_0 - \dfrac{1}{2\gamma} \sum_{i=0}^t v(x_i)) = \pi_Q(x_0 - \dfrac{1}{2\gamma} s_t) $$


 

In [1]:
import numpy as np
import copy

METHOD_QUASI_MONOTONE_DEFAULT_GAMMA = 1.0

In [82]:
def SGMDoubleSimpleAveraging(oracle, projection_function, dimension=0, num_iterations=100, gamma=METHOD_QUASI_MONOTONE_DEFAULT_GAMMA):

    oracle_calls = 0
    iteration_number = 0
    lambda_k = projection_function(np.zeros(dimension, dtype=float))
    s_k = np.zeros(dimension, dtype=float) 


    while iteration_number < num_iterations:
        d_k, diff_d_k = oracle(lambda_k)
        oracle_calls += 1
       
        s_k += diff_d_k
        lambda_k_plus = float(1.0)/float(2*gamma*np.sqrt(iteration_number+1)) * s_k
        lambda_k_plus = projection_function(lambda_k_plus)

        lambda_k = float(iteration_number+1)/float(iteration_number+2)*lambda_k \
                        + float(1.0)/float(iteration_number+2)*lambda_k_plus

        iteration_number += 1
        
    return -d_k, lambda_k

\begin{equation*}
f(x_1,x_2) = 
 \begin{cases}
   5(9x_1^2 +16x_2^2)^{\frac{1}{2}} & x_1 > |x_2| \\
   9x_1 + 16|x_2| & x_1 \le |x_2|
 \end{cases}
\end{equation*}
  $$ s.t. -3 \le x_1,x_2 \le 3$$
 $$\min_{x_1,x_2}f(x_1,x_2) = \max_{\lambda} \min_{x_1,x_2,x_3}L(x_1,x_2,x_3) = \max_{\lambda} \min_{x_1,x_2,x_3}f(x_1,x_2,x_3) = \min_{\lambda} \min_{x_1,x_2,x_3}-f(x_1,x_2,x_3)$$

In [83]:
def oracle(x):

    if x[0] > abs(x[1]):
        f_x = 5*(9*x[0]**2 + 16*x[1]**2)**(float(1)/float(2))
        diff_f_x = np.array([float(9*5*x[0])/np.sqrt(9*x[0]**2 + 16*x[1]**2), 
                             float(16*5*x[0])/np.sqrt(9*x[0]**2 + 16*x[1]**2)])
    else:
        f_x = 9*x[0] + 16*abs(x[1])
        if x[1] >= 0:
            diff_f_x = np.array([9, 16], dtype=float)
        else:
            diff_f_x = np.array([9, -16], dtype=float)
    return  -f_x, -diff_f_x  # return negation to minimize


def projection_function(x):

    return np.array([min(max(x[0],-3),3), min(max(x[1],-3),3)])

In [84]:
SGMDoubleSimpleAveraging(oracle, projection_function, dimension=2, num_iterations=1000)

(-26.023005963338985, array([-2.997003  , -0.05678802]))

Другой пример:
    $$\min_{x}\sum_{i=1}^k f_i \max_{s}((a_{is},x) + c_s)$$

In [104]:
import random
f = [random.uniform(-100, 100),random.uniform(-100, 100),random.uniform(-100, 100),\
    random.uniform(-100, 100),random.uniform(-100, 100)]
a = np.array([[[random.uniform(-100, 100),random.uniform(-100, 100)],[random.uniform(-100, 100),\
    random.uniform(-100, 100)]],[[random.uniform(-100, 100),random.uniform(-100, 100)],\
    [random.uniform(-100, 100),random.uniform(-100, 100)]],[[random.uniform(-100, 100),\
    random.uniform(-100, 100)],[random.uniform(-100, 100),random.uniform(-100, 100)]],\
    [[random.uniform(-100, 100),random.uniform(-100, 100)],[random.uniform(-100, 100),\
    random.uniform(-100, 100)]],[[random.uniform(-100, 100),random.uniform(-100, 100)],\
    [random.uniform(-100, 100),random.uniform(-100, 100)]]])
c = [random.uniform(-100, 100),random.uniform(-100, 100)]

In [107]:
def oracle_2(x):
    diff_f_x = [0,0]
    f_x = 0
    k =0
    m =0
    maxx_i = [0,0,0,0,0]
    maxx = -100000000000;
    while(k<5):
        while(m<2):
            if (a[k][m][0]*x[0] + a[k][m][1]*x[1] - c[m]> maxx):
                maxx = a[k][m][0]*x[0] + a[k][m][1]*x[1] - c[m]
                maxx_i[k] = m 
            m = m +1
           
        f_x =+ f[k]*maxx
        k = k +1
        m = 0
    for i in range(5):
        diff_f_x[0] =+ f[i]*a[i][maxx_i[i]][0]
        diff_f_x[1] =+ f[i]*a[i][maxx_i[i]][1]
    diff_f_x[0] = -diff_f_x[0]
    diff_f_x[1] = -diff_f_x[1] 
    return -f_x, diff_f_x   

In [108]:
SGMDoubleSimpleAveraging(oracle_2, projection_function_2, dimension=2, num_iterations=1000)

(-402348986.79427612, array([ 27892.28106219,  58857.42380908]))

In [6]:
def projection_function_2(x):
    return x;